<a href="https://colab.research.google.com/github/0cleopatra0/therapist-chatbot/blob/main/finetune_peft_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0. Runtime ▸ Change runtime type ▸ GPU, PyTorch 2.x
!pip install -q transformers peft datasets bitsandbytes accelerate einops

BASE_MODEL   = "mistralai/Mistral-7B-Instruct-v0.2"
DATA_PATH    = "/content/mental_health_chat.csv"      # upload via Colab sidebar
NEW_REPO     = "your-huggingface-username/therapist-adapter"

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch, pandas as pd

# 1. Load & tokenize
df   = pd.read_csv(DATA_PATH)
data = load_dataset("csv", data_files=DATA_PATH)["train"]

tok  = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
def tok_fn(row):
    prompt = f"<s>[INST]@Therapist. {row['user_input']} [/INST]"
    row["input_ids"] = tok(prompt, truncation=True, max_length=512).input_ids
    row["labels"]    = tok(row["therapist_response"], truncation=True,
                           max_length=512).input_ids
    return row
data = data.map(tok_fn)

# 2. 4-bit load + LoRA
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)
model = prepare_model_for_kbit_training(model)
peft_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj","v_proj","k_proj","o_proj"]
)
model = get_peft_model(model, peft_cfg)

# 3. Train
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    output_dir         = "./peft-out",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    learning_rate      = 2e-4,
    num_train_epochs   = 1,
    fp16               = True,
    logging_steps      = 25,
    save_total_limit   = 1
)
trainer = Trainer(model=model, args=args, train_dataset=data)
trainer.train()

# 4. Push adapter (tiny, ≈90 MB) to HF Hub
model.push_to_hub(NEW_REPO, use_temp_dir=True)
tok.push_to_hub(NEW_REPO, use_temp_dir=True)
print("Adapter pushed to", NEW_REPO)
